In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#현재경로 확인
os.getcwd()

'c:\\Users\\백원중\\OneDrive\\바탕 화면\\새 폴더'

In [6]:
#데이터 불러오기 (집값 예측)
data = pd.read_csv("./kc_house_data.csv")
data.head() #데이터 확인

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [7]:
ncar=data.shape[0]
nvar=data.shape[1]
print(ncar)
print(nvar)

21613
14


In [10]:
#의미없는 변수 제거
data = data.drop(['id','date','zipcode','lat','long'],axis=1)

In [11]:
feature_colums = list(data.columns.difference(['price'])) #price빼고 나머지 feature_colums에 넣음
X= data[feature_colums]
y=data['price']
train_x, test_x, train_y, test_y = train_test_split(X,y,test_size=0.3, random_state=42) #학습데이터와 평가데이터 7:3
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape) #데이터 개수 확인

(15129, 8) (6484, 8) (15129,) (6484,)


In [12]:
#선형 회귀 모형에 적합
import statsmodels.api as sm
from sklearn.metrics import  mean_squared_error, r2_score
from math import  sqrt

sm_train_x = sm.add_constant(train_x,has_constant="add")
sm_model=sm.OLS(train_y,sm_train_x)
fitted_sm_model=sm_model.fit()
fitted_sm_model.summary()
#bethrooms, bedroom 다중공산성이 있는지 bedroom의 p-value 매우높음

C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.595
Model:                            OLS   Adj. R-squared:                  0.595
Method:                 Least Squares   F-statistic:                     2776.
Date:                Mon, 16 Aug 2021   Prob (F-statistic):               0.00
Time:                        10:06:59   Log-Likelihood:            -2.0826e+05
No. Observations:               15129   AIC:                         4.165e+05
Df Residuals:                   15120   BIC:                         4.166e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         7.186e+06   1.73e+05     41.548      0.000    6.85e+06    7.52e+06
bathrooms     1.303e+05   3960.833     32.889      0.000    1.23e+05    1.38e+05
bedrooms     -2224.7910   2382.356     -0.934      0.350   -6894.497    2444.915
condition     1.641e+04   3169.013      5.178      0.000    1.02e+04    2.26e+04
floors        1946.3052   4336.838      0.449      0.654   -6554.422    1.04e+04
grade         1.956e+05   2199.540     88.924      0.000    1.91e+05       2e+05
waterfront    7.555e+05   2.26e+04     33.479      0.000    7.11e+05       8e+05
yr_built     -4300.7865     88.073    -48.832      0.000   -4473.420   -4128.153
yr_renovated    12.7325      5.043      2.525      0.012       2.847      22.618
==============================================================================
Omnibus:                    13447.374   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1684794.827
Skew:                           3.763   Prob(JB):                         0.00
Kurtosis:                      54.147   Cond. No.                     1.82e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
sm_test_x=sm.add_constant(test_x, has_constant="add")
sm_model_predict=fitted_sm_model.predict(sm_test_x)

C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [14]:
sqrt(mean_squared_error(sm_model_predict,test_y)) #RMSE

239804.29670858165

In [33]:
#Bagging 하기 패키지가 있지만 for문으로 만들기
import random
bagging_predict_result=[]
for _ in range(30):
    data_index=[data_index for data_index in range(train_x.shape[0])]
    random_data_index = np.random.choice(data_index, train_x.shape[0])
    print(len(set(random_data_index)))
    sm_train_x=train_x.iloc[random_data_index,]
    sm_train_y=train_y.iloc[random_data_index,]
    sm_train_x=sm.add_constant(sm_train_x, has_constant="add")
    sm_model=sm.OLS(sm_train_y,sm_train_x)
    fitted_sm_model=sm_model.fit()
    pred=fitted_sm_model.predict(sm_test_x)
    bagging_predict_result.append(pred)
    print(sqrt(mean_squared_error(pred,test_y)))

9542
239664.18800695386
9562
239738.25962861613
9518
239787.09994662294
9553
240051.68138979038
9533
240154.77944614846
9566
240521.542662373
9514
240068.95063336793
9614
239878.5277229924
9607
240612.9849162707


C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except f

9511
240220.96002326746
9496
239863.09311255038
9572
239910.60768420759
9662
239854.73892852632
9535
240667.4835973204
9521
240724.65341733108
9554
240154.23010292658
9565
240489.06036626472
9545
239736.2041618315


C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except f

9565
240345.11215644723
9593
239709.3824079833
9522
240275.3762034818
9524
239818.9810967593
9535
240221.2254226553
9534
240208.5024889569
9545
239731.36241170514
9573
240573.67410719933
9636
239709.7515543837


C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except f

9537
239879.9229159705
9542
240079.2349147257
9538
239891.36308132997


C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Python39\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [27]:
bagging_predict_result[3]

735      5.673722e+05
2830     7.020506e+05
4106     1.116031e+06
16218    1.459717e+06
19964    6.987376e+05
             ...     
12606    6.013725e+05
14393    6.666445e+05
6899     3.277788e+05
85       9.083640e+05
21363    4.416662e+05
Length: 6484, dtype: float64

In [35]:
bagging_predict=[]
for lst2_index in range(test_x.shape[0]):
    temp_predict=[]
    for lst_index in range(len(bagging_predict_result)):
        temp_predict.append(bagging_predict_result[lst_index].values[lst2_index])
    bagging_predict.append(np.mean(temp_predict))

In [36]:
sqrt(mean_squared_error(bagging_predict,test_y))

239862.59277499883